In [1]:
# WARNING: This notebook creates a new excel sheet titled 'polarity_scores_fr.xlsx' and WILL
# replace what is already there, not to mention it could run for several minutes as there is
# a lot of data to process. Do NOT run this notebook unless you are sure of what you are doing.


import newspaper
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd


count_inner = 0
count_outer = 0
broken_links = []
big_dataframe = pd.DataFrame({
    "Article Title": [],
    "Publish Date": [],
    "Clean Link": [],
    "Article URL": [],
    "Categories": [],
    "Sentence Count": [],
    "Sum of Sentence Polarities": [],
    "Average Sentence Polarity": [],
    "Whole Article Polarity": []
})
for sheet in pd.ExcelFile('article_data_fixed.xlsx').sheet_names:
    data = pd.read_excel('article_data_fixed.xlsx', sheet_name=sheet)
    article_titles = []
    publish_dates = []
    clean_links = []
    article_urls = []
    categories = []
    sentence_counts = []
    sum_compound_polarities = []
    average_compound_polarities = []
    whole_commpound_polarities = []
    for i in range(len(data)):
        try:

            url = data.loc[i, "link"]
            article = newspaper.Article(url)
            article.download()
            article.parse()
            # Loads article into newspaper object via URL
            article_titles.append(data.loc[i, "title"])
            publish_dates.append(data.loc[i, "published_date"])
            clean_links.append(data.loc[i, "clean_url"])
            article_urls.append(data.loc[i, "link"])
            categories.append(sheet)


            

            # Defines stopwords (common words such as pronouns and articles that can falsely influence sentiment)
            stopwords = nltk.corpus.stopwords.words("english")

            # Loads article text and tokenizes it into sentences, instantiates sentiment analysis object
            test_par = article.text
            vader = SentimentIntensityAnalyzer()
            test_sentences = sent_tokenize(test_par)
            polarity_scores = []
            sentences_stopped = []

            # Removes stopwords via tokenization and then converts back to string
            for sentence in test_sentences:
                sentence = word_tokenize(sentence)
                new_sentence = [w for w in sentence if w.lower() not in stopwords]
                #print(new_sentence)
                sentence_string = ""
                for word in new_sentence:
                    sentence_string = sentence_string + word + " "
                #print(sentence_string)
                sentences_stopped.append(sentence_string)
                #print(sentence_string)
                polarity_scores.append(vader.polarity_scores(sentence_string))
                #print(vader.polarity_scores(sentence_string))


            # Computes total and average polarity score
            total_polarity = 0
            for score in polarity_scores:
                total_polarity = total_polarity + score['compound']
            #print(total_polarity)
            average_polarity = total_polarity / len(polarity_scores)
            #print("This article contains " + str(len(polarity_scores)) + " sentences.")
            sentence_counts.append(len(polarity_scores))
            #print("The total compound polarity score for this article is: " + str(total_polarity) + ".")
            sum_compound_polarities.append(total_polarity)
            #print("The average compound polarity score for this article is: " + str(average_polarity) + ".")
            average_compound_polarities.append(average_polarity)
            #print("The compound polarity for this whole article is: " + str(vader.polarity_scores(test_par)['compound']) + ".")
            whole_commpound_polarities.append(vader.polarity_scores(test_par)['compound'])

            
            count_inner = count_inner + 1
            print(count_inner)
        except:
            broken_links.append(data.loc[i, "link"])
            continue
    small_dataframe = pd.DataFrame({
                "Article Title": article_titles,
                "Publish Date": publish_dates,
                "Clean Link": clean_links,
                "Article URL": article_urls,
                "Category": categories,
                "Sentence Count": sentence_counts,
                "Sum of Sentence Polarities": sum_compound_polarities,
                "Average Sentence Polarity": average_compound_polarities,
                "Whole Article Polarity": whole_commpound_polarities
            })

    big_dataframe = pd.concat([big_dataframe, small_dataframe])
    count_outer = count_outer + 1
    print(count_outer)
with pd.ExcelWriter("polarity_scores_fr.xlsx", mode='a', if_sheet_exists='replace') as writer:
    big_dataframe.to_excel(writer)

print(broken_links)


1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
2
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
3
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


ValueError: All arrays must be of the same length